# Ex-11 MNIST Dataset Basic Approach

In [ ]:
import pandas as pd
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import random
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import torch

In [ ]:
if torch.cuda. is_available():
    current_device = torch.cuda.current_device()
    torch.cuda.set_device(current_device)
    device = torch.device('cuda:{}'.format(current_device))
    print('Using GPU: {}'.format(torch.cuda.get_device_name(current_device)))
else:
    device = torch.device('cpu')


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
Y = train_df.label.values

In [ ]:
X = train_df.drop(labels='label',axis=1).values / 255

In [ ]:
X

In [ ]:
X_test = test_df.values / 255

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y ,random_state = 42,test_size = 0.2)

In [ ]:
x_test[1]

In [ ]:
x_trainTorch = torch.from_numpy(x_train).type(torch.FloatTensor)
y_trainTorch = torch.from_numpy(y_train).type(torch.FloatTensor)

In [ ]:
x_trainTorch[1]

In [ ]:
x_testTorch = torch.from_numpy(x_test).type(torch.FloatTensor)
y_testTorch = torch.from_numpy(y_test).type(torch.FloatTensor)

In [ ]:
X_testTorch = torch.from_numpy(np.array(X_test)).type(torch.FloatTensor)

In [ ]:
train = torch.utils.data.TensorDataset(x_trainTorch,y_trainTorch)
testval = torch.utils.data.TensorDataset(x_testTorch,y_testTorch)
test = torch.utils.data.TensorDataset(X_testTorch)

In [ ]:
train_loader = DataLoader(train,batch_size=100,shuffle=False)
testval_loader = DataLoader(testval,batch_size=100,shuffle=False)
test_loader = DataLoader(test,batch_size=100,shuffle=False)

In [ ]:
randomlist = []
for i in range(0,9):
  n = random.randint(0,len(X))
  randomlist.append(n)

fig = plt.figure(figsize=(15,8))
gs1 = gridspec.GridSpec(3, 3)
axs = []
for num in range(len(randomlist)):
    axs.append(fig.add_subplot(gs1[num - 1]))
    axs[-1].imshow(X[randomlist[num]].reshape(28,28))
    axs[-1].set_title("Actual: " + str(Y[randomlist[num]]))
fig.subplots_adjust(hspace=0.3)
plt.show()

In [ ]:
randomlist = []
for i in range(0,9):
  n = random.randint(0,len(X))
  randomlist.append(n)

In [ ]:
randomlist

In [ ]:

fig = plt.figure(figsize=(15,8))
gs1 = gridspec.GridSpec(3, 3)
axs = []

In [ ]:
class Net(nn.Module):
  def __init__(self):
        super(Net, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(5,5), stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2))
        self.dropout1 = nn.Dropout(0.2)
        self.c2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.dropout2 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(800, 256)
        self.dropout3 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 10)
        
  def forward(self, x):
        
        output = self.c1(x) 
        output = self.relu1(output) 
        output = self.maxpool1(output)
        output = self.dropout1(output) 
        output = self.c2(output) 
        output = self.relu2(output) 
        output = self.maxpool2(output)
        output = self.dropout2(output) 
        output = output.view(output.size(0), -1) 
        output = self.fc1(output)
        output = self.dropout3(output)
        output = self.fc2(output)
        return output

model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.0002)
criterion = nn.CrossEntropyLoss()


In [ ]:
if torch.cuda.is_available():
    print("Model will be training on GPU")
    model = model.cuda()
    criterion = criterion.cuda()
else:
    print("Model will be training on CPU")

In [ ]:
model.train()
n_iterations = 0
print_every = 50
steps = 0
train_losses, val_losses = [], []
total_epochs = 50
for epoch_number in range(total_epochs):
    running_loss = 0
    for i, (images, labels) in enumerate (train_loader):
        steps += 1
        data, target = Variable(images.view(100,1,28,28)), Variable(labels)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        output = model(data)
        model.zero_grad()
        loss = criterion(output, target)

        n_iterations += 1
        
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if steps % print_every == 0:
            val_loss = 0
            accuracy = 0
            
            
            # Turn off gradients for validation
            with torch.no_grad():
                model.eval()
                for images, labels in val_loader:
                    data, target = Variable(images.view(100,1,28,28), volatile=True), Variable(labels)

                    if torch.cuda.is_available():
                        data = data.cuda()
                        target = target.cuda()
                    
                    log_ps = model(data)
                    loss = criterion(log_ps, target)
                    val_loss += loss.item()

                    ps = torch.exp(log_ps)
                    
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == target.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
        
            model.train()

            train_losses.append(running_loss/len(train_loader))
            val_losses.append(val_loss / len(val_loader))

    print("Epoch: {}/{}.. ".format(epoch_number + 1, total_epochs),
                  "Training Loss: {:.3f}.. ".format(train_losses[-1]),
                  "Val Loss: {:.3f}.. ".format(val_losses[-1]),
                  "Val Accuracy: {:.3f}".format(accuracy/len(val_loader)))

In [ ]:
mnist.train.num_examples

In [ ]:
mnist.test.num_examples

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
mnist.train.images.shape

In [ ]:
mnist.train.images[1]

In [ ]:
single_img = mnist.train.images[1].reshape(28,28)

In [ ]:
plt.imshow(single_img,cmap="gist_gray")

In [ ]:
single_img.min()

In [ ]:
single_img.max()

In [ ]:
#y =mx + b

x = tf.placeholder(tf.float32,shape=[None,784])

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
y = tf.matmul(x,W) + b

In [ ]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [ ]:
#LOSS Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y))

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)

In [ ]:
train = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1000):
        
        batch_x,batch_y = mnist.train.next_batch(100)
        
        sess.run(train,feed_dict = {x:batch_x,y_true:batch_y})
        
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_true,1))

    acc = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    print(sess.run(acc,feed_dict={x:mnist.test.images, y_true:mnist.test.labels}))
    